In [2]:
from utils.data_loader import load_for_mlp
import tensorflow as tf
import numpy as np
from joblib import load

In [6]:
model_path = "../../outputs/mlp/2025-09-23_16-04"    
dataset_id = "exp1"

# load model
model = tf.keras.models.load_model(model_path + "/model.keras")

# load and prepare data
X_test_flat, Y_test_flat = load_for_mlp(start_year=2014, end_year=2015, datasets=[dataset_id])

mask = X_test_flat[:, 10] == 1
X_test_flat = np.delete(X_test_flat, [10], axis=1)

scaler = load(model_path + '/scaler.pkl')
n_samples, n_features = X_test_flat.shape
X_test_scaled_flat = scaler.transform(X_test_flat)

map_samples = int(n_samples/167/360)

map_mask = mask.reshape(map_samples, 167, 360)[0, :, : ]
X_test = X_test_scaled_flat.reshape(map_samples, 167, 360, n_features)


In [7]:
from utils.model_analysis import plot_shap_over_time

# plot feature importance over time per basin

feature_names = ['SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 'water_flux_up', 'stress_X', 'stress_Y', 'currents_X', 'currents_Y','lat','nav_lon_sin','nav_lon_cos','month-sin','month-cos']

mean_abs_shap, names = plot_shap_over_time(
    model, X_test, map_mask, feature_names=feature_names,
    bg_per_time=20,            # small background per time step
    max_samples_per_time=50, # subsample for speed (optional)
    batch_size=4096,
    region_id='ARCTIC',
    type='SEASONAL'
)

12


PermutationExplainer explainer: 51it [00:30,  1.22it/s]                        


Processed time step 1/12


KeyboardInterrupt: 